# Trends in Caribbean atmospheric stability

Plot trends in convective available potential energy (CAPE), convective inhibition (CIN) and 300-hPa temperatures over the Caribbean and tropical North Atlantic regions.

In [ ]:
# Package imports
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.mpl.ticker import LatitudeFormatter, LongitudeFormatter
import cmaps as ncl_cm

from cartopy.mpl.geoaxes import GeoAxes
from mpl_toolkits.axes_grid1 import AxesGrid
import matplotlib.ticker as mticker


In [ ]:
# Figure format defaults
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['figure.titlesize'] = 15
plt.rcParams["axes.titlesize"] = 15

# Define AxesGrid parameters to set map projections and orientation
projection = ccrs.PlateCarree(central_longitude=-180)
mproj = ccrs.PlateCarree()
axes_class = (GeoAxes, dict(projection=projection))

## Retrieve trend data
Trend data is generated using the ECMWF fifth generation reanalysis (ERA5) with 2.5°x2.5° horizontal grid resolution.

In [ ]:
# Seasonal trend fields
sst_trend_results = xr.open_dataset('../data/sst_MJJ_trend_results_1979to2024.nc')
temp_trend_results = xr.open_dataset('../data/temp300_MJJ_trend_results_1979to2024.nc')
cape_trend_results = xr.open_dataset('../data/cape_MJJ_trend_results_1979to2024.nc')
cin_trend_results = xr.open_dataset('../data/cin_MJJ_trend_results_1979to2024.nc')


## Plot spatial trend maps

Also, we calculate the ratio between $\frac{SST_{trend}}{T300_{trend}}$ to assess the rate of warming aloft relative to warming at the surface.

In [ ]:
import matplotlib.gridspec as gridspec
import matplotlib.colors as colors

fig = plt.figure(figsize=(14, 8), dpi=300)
gs = gridspec.GridSpec(2, 2, figure=fig)

# Define subplots
ax1 = fig.add_subplot(gs[0, 0], projection=ccrs.PlateCarree())
ax2 = fig.add_subplot(gs[0, 1], projection=ccrs.PlateCarree())
ax3 = fig.add_subplot(gs[1, 0], projection=ccrs.PlateCarree())
ax4 = fig.add_subplot(gs[1, 1], projection=ccrs.PlateCarree())


# --- Subplot 1 ---
# Set the extent for each subplot
ax1.set_extent([-100, -20, 5, 25])
ax1.add_feature(cfeature.COASTLINE, linewidth=0.5, zorder=2, color='black')
ax1.set_xticks(np.arange(-90, -20, 10), crs=ccrs.PlateCarree())
ax1.set_yticks(np.arange(5, 35, 5), crs=ccrs.PlateCarree())
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax1.xaxis.set_major_formatter(lon_formatter)
ax1.yaxis.set_major_formatter(lat_formatter)
ax1.tick_params(axis='both', which='major', labelsize=14)

levels = np.arange(-8, 8.5, 0.5)
im = ax1.contourf(cape_trend_results.longitude.values,
                  cape_trend_results.latitude.values,
                  cape_trend_results.trend.values[:, :],
                  levels=levels,
                  cmap=ncl_cm.BlueWhiteOrangeRed, extend='both', transform=ccrs.PlateCarree())

# Mesh the entire grid
fill = ax1.fill_between(ax1.get_xlim(), *ax1.get_ylim(), color='none', edgecolor='black', lw=0, hatch=3*'/', alpha=0.5)

# Mask trend where p_value is less than 0.05
all_trends_revmask = np.where(cape_trend_results.p_value.values < 0.05, np.nan, cape_trend_results.trend.values)

# Plot the data for the current month
im = ax1.contourf(cape_trend_results.longitude.values,
                  cape_trend_results.latitude.values,
                  all_trends_revmask[:, :], levels=levels,
                  cmap=ncl_cm.BlueWhiteOrangeRed, extend='both', transform=ccrs.PlateCarree())

# Subplot titles
ax1.set_title('a) CAPE trend', loc='left')

# Add a single colorbar for all subplots
cb = plt.colorbar(im, orientation='horizontal')
cb.set_label('(J kg$^{-1}$ yr$^{-1}$)', fontsize=12)

# --- Subplot 2 ---
# Set the extent for each subplot
ax2.set_extent([-100, -20, 5, 25])
ax2.add_feature(cfeature.COASTLINE, linewidth=0.5, zorder=2, color='black')
ax2.set_xticks(np.arange(-90, -20, 10), crs=ccrs.PlateCarree())
ax2.set_yticks(np.arange(5, 35, 5), crs=ccrs.PlateCarree())
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax2.xaxis.set_major_formatter(lon_formatter)
ax2.yaxis.set_major_formatter(lat_formatter)
ax2.tick_params(axis='both', which='major', labelsize=14)

# Plot the data for the current month
levels = np.arange(-5, 5.5, 0.5)
im = ax2.contourf(cin_trend_results.longitude.values,
                  cin_trend_results.latitude.values,
                  cin_trend_results.trend.values[:, :],
                  levels=levels,
                  cmap=ncl_cm.BlueWhiteOrangeRed, extend='both',
                  transform=ccrs.PlateCarree())

# Mesh the entire grid
fill = ax2.fill_between(ax2.get_xlim(), *ax2.get_ylim(), color='none', edgecolor='black', lw=0, hatch=3*'/', alpha=0.5)

# Mask extreme values in CIN, these may be boundary errors (?)



# Mask trend where p_value is less than 0.05
all_trends_revmask = np.where(cin_trend_results.p_value.values < 0.05, np.nan, cin_trend_results.trend.values)

# Plot the data for the current month
im = ax2.contourf(cin_trend_results.longitude.values,
                  cin_trend_results.latitude.values,
                  all_trends_revmask[:, :],
                  levels=levels,
                  cmap=ncl_cm.BlueWhiteOrangeRed, extend='both',
                  transform=ccrs.PlateCarree())

# Subplot titles
ax2.set_title('b) CIN trend', loc='left')

# Add a single colorbar for all subplots
cb = plt.colorbar(im, orientation='horizontal')
cb.set_label('(kg$^{-1}$ yr$^{-1}$)', fontsize=12)

# --- Subplot 3 ---
ax3.set_extent([-100, -20, 5, 25])
ax3.add_feature(cfeature.COASTLINE, linewidth=0.5, zorder=2, color='black')
ax3.set_xticks(np.arange(-90, -20, 10), crs=ccrs.PlateCarree())
ax3.set_yticks(np.arange(5, 35, 5), crs=ccrs.PlateCarree())
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax3.xaxis.set_major_formatter(lon_formatter)
ax3.yaxis.set_major_formatter(lat_formatter)
ax3.tick_params(axis='both', which='major', labelsize=14)

# Plot the data for the current month
levels = np.arange(-0.024, 0.028, 0.004)
im = ax3.contourf(temp_trend_results.longitude.values,
                  temp_trend_results.latitude.values,
                  temp_trend_results.trend.values[:, :],
                  levels=levels,
                  cmap=ncl_cm.BlueWhiteOrangeRed, extend='both', transform=ccrs.PlateCarree())

# Mesh the entire grid
fill = ax3.fill_between(ax3.get_xlim(), *ax3.get_ylim(), color='none', edgecolor='black', lw=0, hatch=3*'/', alpha=0.5)

# Mask trend where p_value is less than 0.05
all_trends_revmask = np.where(temp_trend_results.p_value.values < 0.05, np.nan, temp_trend_results.trend.values)

# Plot the data for the current month
im = ax3.contourf(temp_trend_results.longitude.values,
                  temp_trend_results.latitude.values,
                  all_trends_revmask[:, :],
                  levels=levels,
                  cmap=ncl_cm.BlueWhiteOrangeRed, extend='both', transform=ccrs.PlateCarree())

# Subplot titles
ax3.set_title('c) T300 trend', loc='left')

# Add a single colorbar for all subplots
cb = plt.colorbar(im, orientation='horizontal')
cb.set_label('(°C yr$^{-1}$)', fontsize=12)

# --- Subplot 4 ---
ax4.set_extent([-100, -20, 5, 25])
ax4.add_feature(cfeature.COASTLINE, linewidth=0.5, zorder=4)
ax4.add_feature(cfeature.LAND, zorder=3, color='lightgray')
ax4.set_xticks(np.arange(-90, -20, 10), crs=ccrs.PlateCarree())
ax4.set_yticks(np.arange(5, 35, 5), crs=ccrs.PlateCarree())
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax4.xaxis.set_major_formatter(lon_formatter)
ax4.yaxis.set_major_formatter(lat_formatter)
ax4.tick_params(axis='both', which='major', labelsize=14)

# Plot the data for the current month
levels = np.arange(0, 1.6, 0.1)
im = ax4.contourf(sst_temp_ratio.longitude.values,
                  sst_temp_ratio.latitude.values,
                  sst_temp_ratio.values[:, :],
                  levels=levels,
                  cmap=ncl_cm.WhiteBlueGreenYellowRed,
                  extend='both', transform=ccrs.PlateCarree())

# Add contour to indicate a ratio of 1
cntr = ax4.contour(sst_temp_ratio.longitude.values,
                  sst_temp_ratio.latitude.values,
                  sst_temp_ratio.values[:, :],
                  levels=[1],
                  linewidth=2,
                  colors='black',
                  transform=ccrs.PlateCarree(),
                   zorder=3,
                  )

# Contour label
cntr.clabel(fmt='%1.1f', fontsize=8)

# Add domain box for CLLJ
rect = plt.Rectangle((-80, 13), 10, 4, linewidth=2, edgecolor='r', facecolor='none', zorder=4)
ax4.add_patch(rect)

# Subplot titles
ax4.set_title('d) $SST_{trend}/T300_{trend}$ ratio', loc='left')

# Add a single colorbar for all subplots
cb = plt.colorbar(im, orientation='horizontal')
cb.set_label(' ', fontsize=12)

# --- Save figure ---
fig.savefig('../figures/figure3_atmospheric_stability.pdf', dpi=300, format='pdf', bbox_inches='tight')